# TFX Interactive Development

In [40]:
import absl
import os
import tempfile

import tensorflow as tf
import tensorflow_data_validation as tfdv
import tfx

from pprint import pprint
from tensorflow_metadata.proto.v0 import schema_pb2
from tfx.components import CsvExampleGen
from tfx.components import Evaluator
from tfx.components import ExampleValidator
from tfx.components import ModelValidator
from tfx.components import Pusher
from tfx.components import SchemaGen
from tfx.components import StatisticsGen
from tfx.components import Trainer
from tfx.components import Transform
from tfx.components.common_nodes.importer_node import ImporterNode
from tfx.orchestration import metadata
from tfx.orchestration import pipeline
from tfx.orchestration.experimental.interactive.interactive_context import InteractiveContext
from tfx.proto import evaluator_pb2
from tfx.proto import example_gen_pb2
from tfx.proto import pusher_pb2
from tfx.proto import trainer_pb2
from tfx.proto.evaluator_pb2 import SingleSlicingSpec
from tfx.utils.dsl_utils import external_input


print("Tensorflow Version:", tf.__version__)
print("TFX Version:", tfx.__version__)

Tensorflow Version: 1.15.0
TFX Version: 0.15.0


## Configure lab settings

Set location paths, connections strings, and other environment settings. Make sure to update project id, region, and artifact store GCS bucket with your settings.

In [41]:
PROJECT_ID = 'mlops-workshop'
REGION = 'us-central1'
ARTIFACT_STORE='gs://mlops-workshop-artifact-store'

DATASET_GCS_PATH = 'gs://workshop-datasets/covertype/full/'
DATASET_WITH_ANOMALIES_GCS_PATH='gs://workshop-datasets/covertype/evaluation_anomalies/'

## TFX Component Walk-through

### Create Interactive Context

TFX Interactive Context allows you to create and run TFX Components in an interactive mode. It is designed to support experimentation and development in a Jupyter Notebook environment. It is an experimental feature and major changes to interface and functionality are expected. When creating the interactive context you can specifiy the following parameters:
- `pipeline_name` - Optional name of the pipeline for ML Metadata tracking purposes. If not specified, a name will be generated for you.
- `pipeline_root` - Optional path to the root of the pipeline's outputs. If not specified, an ephemeral temporary directory will be created and used.
- `metadata_connection_config` - Optional metadata_store_pb2.ConnectionConfig instance used to configure connection to a ML Metadata connection. If not specified, an ephemeral SQLite MLMD connection contained in the pipeline_root directory with file name "metadata.sqlite" will be used.


In [42]:
PIPELINE_NAME='tfx-covertype-classifier'
PIPELINE_ROOT='/home/tfx_pipeline_root'

os.makedirs(PIPELINE_ROOT, exist_ok=True)

In [43]:
context = InteractiveContext(
    pipeline_name=PIPELINE_NAME,
    pipeline_root=PIPELINE_ROOT,
    metadata_connection_config=None
)

### Ingest data using ExampleGen

The ExampleGen TFX Pipeline component ingests data into TFX pipelines. It consumes external files/services to generate Examples which will be read by other TFX components. It can also split and shuffle the data.

- Consumes: Data from external data sources such as CSV, TFRecord and BigQuery
- Emits: tf.Example records

#### Configure ExampleGen to divide data into training and evaluation splits

In [44]:
output_config = example_gen_pb2.Output(
    split_config=example_gen_pb2.SplitConfig(splits=[
        example_gen_pb2.SplitConfig.Split(name='train', hash_buckets=4),
        example_gen_pb2.SplitConfig.Split(name='eval', hash_buckets=1)
    ]))


example_gen = tfx.components.CsvExampleGen(
    instance_name='Data_Extraction_Spliting',
    input=external_input(DATASET_GCS_PATH),
    output_config=output_config
)

#### Run ExampleGen

In [45]:
context.run(example_gen)

ExecutionResult(
    component_id: CsvExampleGen.Data_Extraction_Spliting
    execution_id: 1
    outputs:
        examples: Channel(
            type_name: ExamplesPath
            artifacts: [Artifact(type_name: ExamplesPath, uri: /home/tfx_pipeline_root/CsvExampleGen.Data_Extraction_Spliting/examples/1/train/, split: train, id: 2)
            Artifact(type_name: ExamplesPath, uri: /home/tfx_pipeline_root/CsvExampleGen.Data_Extraction_Spliting/examples/1/eval/, split: eval, id: 3)]
        ))

### Generate statistics using StatisticsGen

The StatisticsGen  component generates data statistics that can be used by other TFX components. StatisticsGen uses [TensorFlow Data Validation](https://www.tensorflow.org/tfx/data_validation/get_started).

- Consumes: datasets created by ExampleGen 
- Emits: Dataset statistics

#### Run StatisticsGen

In [46]:
statistics_gen = tfx.components.StatisticsGen(
    instance_name='Statistics_Generation',
    examples=example_gen.outputs['examples'])

context.run(statistics_gen)

ExecutionResult(
    component_id: StatisticsGen.Statistics_Generation
    execution_id: 2
    outputs:
        output: Channel(
            type_name: ExampleStatisticsPath
            artifacts: [Artifact(type_name: ExampleStatisticsPath, uri: /home/tfx_pipeline_root/StatisticsGen.Statistics_Generation/output/2/train/, split: train, id: 4)
            Artifact(type_name: ExampleStatisticsPath, uri: /home/tfx_pipeline_root/StatisticsGen.Statistics_Generation/output/2/eval/, split: eval, id: 5)]
        ))

#### Display statistics

In [47]:
context.show(statistics_gen.outputs['statistics'])

### Infer data schema using SchemaGen

Some TFX components use a description of your input data called a schema. The schema is an instance of `schema.proto`. It can specify data types for feature values, whether a feature has to be present in all examples, allowed value ranges, and other properties. `SchemaGen` automatically generates the schema by inferring types, categories, and ranges from data statistics. The auto-generated schema is best-effort and only tries to infer basic properties of the data. It is expected that developers review and modify it as needed. `SchemaGen` uses [TensorFlow Data Validation](https://www.tensorflow.org/tfx/data_validation/get_started).

- Consumes: statistics from `StatisticsGen`
- Emits: Data schema proto

#### Run SchemaGen

In [48]:
schema_gen = SchemaGen(
    statistics=statistics_gen.outputs['statistics'],
    infer_feature_shape=False)

context.run(schema_gen)

ExecutionResult(
    component_id: SchemaGen
    execution_id: 3
    outputs:
        output: Channel(
            type_name: SchemaPath
            artifacts: [Artifact(type_name: SchemaPath, uri: /home/tfx_pipeline_root/SchemaGen/output/3/, split: , id: 6)]
        ))

#### Visualize the inferred schema

In [49]:
context.show(schema_gen.outputs['schema'])

,Type,Presence,Valency,Domain
Feature name,,,,
'Hillshade_9am',INT,required,single,-
'Aspect',INT,required,single,-
'Slope',INT,required,single,-
'Horizontal_Distance_To_Roadways',INT,required,single,-
'Hillshade_Noon',INT,required,single,-
'Elevation',INT,required,single,-
'Horizontal_Distance_To_Fire_Points',INT,required,single,-
'Soil_Type',INT,required,single,-
'Wilderness_Area',STRING,required,single,'Wilderness_Area'


,Values
Domain,
'Wilderness_Area',"'Cache', 'Commanche', 'Neota', 'Rawah'"


In general, TFDV (used by `SchemaGen`) uses conservative heuristics to infer stable data properties from the statistics in order to avoid overfitting the schema to a specific dataset. It is strongly advised to review the inferred schema and refine it as needed, to capture any domain knowledge about the data that TFDV's heuristics might have missed.

In our case `SchemaGen` did not interpret the `Soil_Type` and `Cover_Type` fields properly. Although both fields are encoded as integers, they should be interpreted as categorical rather than numeric.

You can manually update the schema. The `lab-31-tfdv-structure-data` lab dives deeper into the capabilities of TFDV and walks you through an interactive process of discovering and fine tuning the schema for the `covertype` dataset used in this lab.

#### Update the schema

Load the schema from the schema proto file

In [50]:
schema_proto_path = '{}/{}'.format(schema_gen.outputs['schema'].get()[0].uri, 'schema.pbtxt')
schema = tfdv.load_schema_text(schema_proto_path)

Update the `Soil_Type` and `Cover_Type` features.

In [57]:
#soil_type_domain = [
#    '7745', '7202', '7756', '7757', '7201', '4703', '7746', '4744', 
#    '7755', '7700', '4758', '8771', '8772', '4704', '2705', '7102', 
#    '8776', '2703', '2717', '2704', '7101', '6102', '2702', '6101', 
#    '7702', '8703', '6731', '2706', '7790', '4201', '7709', '7710', 
#    '7103', '5101', '7701', '8708', '3502', '8707', '3501', '5151'
#]

soil_type_domain = [
    b'7745', b'7202', b'7756', b'7757', b'7201', b'4703', b'7746', b'4744'
]

tfdv.get_feature(schema, 'Soil_Type').type = schema_pb2.FeatureType.BYTES
tfdv.set_domain(schema, 'Soil_Type', schema_pb2.StringDomain(name='Soil_Type', value=soil_type_domain))

tfdv.set_domain(schema, 'Cover_Type', schema_pb2.IntDomain(name='Cover_Type', min=1, max=7, is_categorical=True))

tfdv.display_schema(schema=schema)

,Type,Presence,Valency,Domain
Feature name,,,,
'Hillshade_9am',INT,required,single,-
'Aspect',INT,required,single,-
'Slope',INT,required,single,-
'Horizontal_Distance_To_Roadways',INT,required,single,-
'Hillshade_Noon',INT,required,single,-
'Elevation',INT,required,single,-
'Horizontal_Distance_To_Fire_Points',INT,required,single,-
'Soil_Type',STRING,required,single,'Soil_Type'
'Wilderness_Area',STRING,required,single,'Wilderness_Area'


,Values
Domain,
'Wilderness_Area',"'Cache', 'Commanche', 'Neota', 'Rawah'"
'Soil_Type',"'7745', '7202', '7756', '7757', '7201', '4703', '7746', '4744'"


Save the updated schema to a proto file.

In [58]:
schema_dir = '/home/covertype_schema'
tf.io.gfile.makedirs(schema_dir)
schema_file = os.path.join(schema_dir, 'schema.pbtxt')

tfdv.write_schema_text(schema, schema_file)

!cat {schema_file}

feature {
  name: "Hillshade_9am"
  value_count {
    min: 1
    max: 1
  }
  type: INT
  presence {
    min_fraction: 1.0
    min_count: 1
  }
}
feature {
  name: "Aspect"
  value_count {
    min: 1
    max: 1
  }
  type: INT
  presence {
    min_fraction: 1.0
    min_count: 1
  }
}
feature {
  name: "Slope"
  value_count {
    min: 1
    max: 1
  }
  type: INT
  presence {
    min_fraction: 1.0
    min_count: 1
  }
}
feature {
  name: "Horizontal_Distance_To_Roadways"
  value_count {
    min: 1
    max: 1
  }
  type: INT
  presence {
    min_fraction: 1.0
    min_count: 1
  }
}
feature {
  name: "Hillshade_Noon"
  value_count {
    min: 1
    max: 1
  }
  type: INT
  presence {
    min_fraction: 1.0
    min_count: 1
  }
}
feature {
  name: "Elevation"
  value_count {
    min: 1
    max: 1
  }
  type: INT
  presence {
    min_fraction: 1.0
    min_count: 1
  }
}
feature {
  name: "Horizontal_Distance_To_Fire_Points"
  value_count {
    min: 1
    max: 1
  }
  type: INT
  presence {
  

### Import the schema using ImporterNode

The `ImporterNode` component allows you to import an external artifact, including the schema file, so it can be used by other TFX components in your workflow. 


#### Import the schema file

In [59]:
schema_importer = ImporterNode(
    instance_name='Schema_Importer',
    source_uri=schema_dir,
    artifact_type=tfx.types.standard_artifacts.Schema,
    reimport=False
)

context.run(schema_importer)

ExecutionResult(
    component_id: ImporterNode.Schema_Importer
    execution_id: 6
    outputs:
        result: Channel(
            type_name: SchemaPath
            artifacts: [Artifact(type_name: SchemaPath, uri: /home/covertype_schema, split: , id: 7)]
        ))

#### Visualize the imported schema

In [60]:
context.show(schema_importer.outputs['result'])

,Type,Presence,Valency,Domain
Feature name,,,,
'Hillshade_9am',INT,required,single,-
'Aspect',INT,required,single,-
'Slope',INT,required,single,-
'Horizontal_Distance_To_Roadways',INT,required,single,-
'Hillshade_Noon',INT,required,single,-
'Elevation',INT,required,single,-
'Horizontal_Distance_To_Fire_Points',INT,required,single,-
'Soil_Type',STRING,required,single,'Soil_Type'
'Wilderness_Area',STRING,required,single,'Wilderness_Area'


,Values
Domain,
'Wilderness_Area',"'Cache', 'Commanche', 'Neota', 'Rawah'"
'Soil_Type',"'7745', '7202', '7756', '7757', '7201', '4703', '7746', '4744'"


### Validate data with ExampleValidator

The `ExampleValidator` component identifies anomalies in data.  It identifies anomalies by comparing data statistics computed by the `StatisticsGen` component against a schema. 

`ExampleValidator` can detect different classes of anomalies. For example it can:

- perform validity checks by comparing data statistics against a schema 
- detect training-serving skew by comparing training and serving data.
- detect data drift by looking at a series of data.


- Consumes: A schema from `SchemaGen` or `ImporterNode`, and statistics from `StatisticsGen`.
- Emits: Validation results

#### Run ExampleValidator


In [61]:
example_validator = ExampleValidator(
    statistics=statistics_gen.outputs['statistics'],
    schema=schema_importer.outputs['result'],
    instance_name="Data_Validation"
)

context.run(example_validator, enable_cache=False)

ExecutionResult(
    component_id: ExampleValidator.Data_Validation
    execution_id: 7
    outputs:
        output: Channel(
            type_name: ExampleValidationPath
            artifacts: [Artifact(type_name: ExampleValidationPath, uri: /home/tfx_pipeline_root/ExampleValidator.Data_Validation/output/7/, split: , id: 9)]
        ))

#### Visualize validation results

In [62]:
context.show(example_validator.outputs['anomalies'])

,Anomaly short description,Anomaly long description
Feature name,,
'Soil_Type',Expected data of type: BYTES but got INT,
